### Step 1. 데이터 다운로드

### Step 2. 데이터 읽어오기

In [1]:
import glob
import os

txt_file_path = '/aiffel/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = [] 

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [2]:
# enumerate() 함수를 이용하여 raw_corpus list 내에 저장된 문장과 그 문장의 인덱스를 반환 (인덱스, 문장 순)
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 문장 10개만 확인
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


### Step 3. 데이터 정제

In [3]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if len(sentence.split(' ')) <= 1: continue
    if len(sentence.split(' ')) > 15: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    

print(len(corpus))
corpus[:10]

164880


['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>',
 '<start> you saw her bathing on the roof <end>',
 '<start> her beauty and the moonlight overthrew her <end>']

### Step 4. 평가 데이터셋 분리

In [5]:
import tensorflow as tf

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

In [6]:
tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2686 ...    0    0    0]
 [   2   34    7 ...    0    0    0]
 ...
 [   2  258  193 ...    0    0    0]
 [   2  132    5 ...    0    0    0]
 [   2    7   33 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fb88ceb59a0>


In [7]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  306   62   57    9  952 5678    3    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
[  50    5   91  306   62   57    9  952 5678    3    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


In [8]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                          tgt_input,
                                          test_size = 0.2,
                                          random_state=7)

### Step 5. 인공지능 만들기

In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 32), (256, 32)), types: (tf.int32, tf.int32)>

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 32, 12001), dtype=float32, numpy=
array([[[-1.72430682e-05,  2.67380506e-06,  5.17131703e-05, ...,
          5.23548260e-05, -5.34188584e-05, -1.98627295e-05],
        [ 1.41809127e-04, -2.26052507e-04, -1.61259431e-05, ...,
          1.09075678e-04, -1.04522820e-04, -1.14310140e-04],
        [ 2.65123323e-04, -4.68807557e-04,  8.91174350e-05, ...,
         -5.62138564e-04,  3.19409664e-05, -8.05863965e-05],
        ...,
        [ 4.44197183e-04, -1.24111166e-03,  1.56346976e-03, ...,
         -4.01407434e-03,  1.94797688e-03, -6.13773195e-03],
        [ 4.35811002e-04, -1.26229285e-03,  1.57259626e-03, ...,
         -4.06635227e-03,  1.97261502e-03, -6.17369637e-03],
        [ 4.28528845e-04, -1.28011953e-03,  1.58036838e-03, ...,
         -4.11162479e-03,  1.99474767e-03, -6.20317180e-03]],

       [[-1.72430682e-05,  2.67380506e-06,  5.17131703e-05, ...,
          5.23548260e-05, -5.34188584e-05, -1.98627295e-05],
        [-1.13755443e-04,  1.23755308e-04,  2

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [13]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
644/644 [==============================] - 274s 422ms/step - loss: 1.6805
Epoch 2/10
644/644 [==============================] - 273s 424ms/step - loss: 1.4057
Epoch 3/10
644/644 [==============================] - 273s 424ms/step - loss: 1.3232
Epoch 4/10
644/644 [==============================] - 274s 425ms/step - loss: 1.2579
Epoch 5/10
644/644 [==============================] - 274s 426ms/step - loss: 1.2017
Epoch 6/10
644/644 [==============================] - 275s 426ms/step - loss: 1.1511
Epoch 7/10
644/644 [==============================] - 274s 426ms/step - loss: 1.1045
Epoch 8/10
644/644 [==============================] - 274s 425ms/step - loss: 1.0612
Epoch 9/10
644/644 [==============================] - 274s 425ms/step - loss: 1.0212
Epoch 10/10
644/644 [==============================] - 274s 426ms/step - loss: 0.9838


In [14]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [15]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so much <end> '

In [16]:
generate_text(model, tokenizer, init_sentence="<start> i will", max_len=20)

'<start> i will never let you go <end> '

<ol>
<ul> 1. <strong>프로젝트 설명</strong>
    <p> 노래 가사를 학습 데이터로 사용하여 모델을 학습시키고 노래 가사를 출력하는 딥러닝 모델을 만들어보았다.</p>
</ul>
<ul> 2. <strong>시도한 것들</strong>
    <li> 데이터 전처리 전에 학습 데이터에 Hallelujah만 반복해서 출력되는 것을 확인하고 토큰의 개수를 1개 이상으로 조정하였다.</li>
    <li> 노래 가사를 만드는게 목적이기 때문에 문장부호를 제외한 토큰의 개수를 15개 이하로 제한하였다.</li>
</ul>
<ul> 3. <strong>알게된 점</strong>
    <li> 토큰을 15개 이하로 제한하는 과정에서 .split()을 sentence에 할 것인지 preprocessed_sentence에 할 것인지에 따라 데이터의 양이 달라졌다. 전처리 하기 전의 문장을 띄워쓰기 기준으로 나누면 문자열과 문장부호가 하나로 합쳐서 하나의 어절을 이룬다. 그러나 전처리 후의 문장을 띄워쓰기를 기준으로 끊으면 문장부호가 독립된 하나의 어절처럼 취급되어 토큰 15개가 넘어가는 경우가 있었다. 따라서 토큰 개수 제한을 하려는 상황에서는 전처리 전에 길이 제한을 하면 더 많은 데이터를 가지고 학습을 할 수 있다. </li>
</ul>
<ul> 4. <strong>어려웠던 점</strong>
    <li> 자연어처리에서는 소스 문장, 타겟 문장을 각각 train data, test data로 사용하였다. 이와 관련하여 토큰화를 진행한 후 끝 단어 end를 없애면 소스 문장, 첫 단어 start를 없애면 타겟 문장으로 사용한다. 그런데 train data를 만들 때 end를 없애는게 아니라 pad일 수도 있는 마지막 토큰 하나를 제거한다. 왜 end를 제거하는 것이 아니라 토큰 길이만 하나 작은 데이터를 test data로 사용하는지 잘 모르겠다.</li>
</ul>
<ul> 5. <strong>자기 다짐</strong>
    <li>RNN에서는 최종 입력되는 값이 결과값에 영향을 미치기 때문에 padding에서 pre방식이 더 유리하다고 한다. 이번 프로젝트에서는 padding을 문장 끝(post 방식)에 주었는데, 다음 번에는 문장 앞에 padding을 주면서 두 가지 결과를 비교해보는 것도 좋을 것 같다.</li>
</ul>
</ol>